In [ ]:

!pip install torch
!pip install scikit-learn

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/torch-2.5.1+cu121.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install torch torchvision torchaudio --upgrade --quiet


In [14]:
# Install required libraries if not already installed
# pip install transformers datasets torch scikit-learn pandas

import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score

import torch
# Load train and test datasets
train_data = pd.read_csv("/content/train.csv")  # Ensure 'Query' and 'Code' columns are present
test_data = pd.read_csv("/content/test.csv")

# Initialize T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")  # Lightweight T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize dataset
def preprocess(data, tokenizer, max_len=128):
    inputs = tokenizer(
        ["generate pandas code: " + query for query in data["Query"].tolist()],
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    targets = tokenizer(
        data["Code"].tolist(),
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    return inputs, targets

train_inputs, train_targets = preprocess(train_data, tokenizer)
test_inputs, test_targets = preprocess(test_data, tokenizer)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=4e-3)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# Training loop
# 50 epoch
epochs = 50
batch_size = 16

for epoch in range(epochs):
    model.train()
    for i in range(0, len(train_inputs["input_ids"]), batch_size):
        input_batch = {key: val[i:i+batch_size].to(device) for key, val in train_inputs.items()}
        target_batch = train_targets["input_ids"][i:i+batch_size].to(device)

        # Forward pass
        outputs = model(**input_batch, labels=target_batch)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Batch {i//batch_size+1}, Loss: {loss.item()}")

Epoch 1, Batch 28, Loss: 0.22443602979183197
Epoch 2, Batch 28, Loss: 0.11626393347978592
Epoch 3, Batch 28, Loss: 0.0922190248966217
Epoch 4, Batch 28, Loss: 0.0426437146961689
Epoch 5, Batch 28, Loss: 0.03998782858252525
Epoch 6, Batch 28, Loss: 0.02549600601196289
Epoch 7, Batch 28, Loss: 0.027540596202015877
Epoch 8, Batch 28, Loss: 0.01483921054750681
Epoch 9, Batch 28, Loss: 0.009183874353766441
Epoch 10, Batch 28, Loss: 0.005429213400930166
Epoch 11, Batch 28, Loss: 0.013458655215799809
Epoch 12, Batch 28, Loss: 0.003113278653472662
Epoch 13, Batch 28, Loss: 0.020267527550458908
Epoch 14, Batch 28, Loss: 0.01666376367211342
Epoch 15, Batch 28, Loss: 0.005814643111079931
Epoch 16, Batch 28, Loss: 0.0031590594444423914
Epoch 17, Batch 28, Loss: 0.002472558058798313
Epoch 18, Batch 28, Loss: 0.0008822854142636061
Epoch 19, Batch 28, Loss: 0.0026547980960458517
Epoch 20, Batch 28, Loss: 0.0023968832101672888
Epoch 21, Batch 28, Loss: 0.0004921859945170581
Epoch 22, Batch 28, Loss: 0

In [16]:
# Evaluation
model.eval()
predictions = []
references = []

with torch.no_grad():
    for i in range(0, len(test_inputs["input_ids"]), batch_size):
        input_batch = {key: val[i:i+batch_size].to(device) for key, val in test_inputs.items()}
        output = model.generate(**input_batch)
        decoded_preds = tokenizer.batch_decode(output, skip_special_tokens=True)
        decoded_refs = tokenizer.batch_decode(test_targets["input_ids"][i:i+batch_size], skip_special_tokens=True)
        predictions.extend(decoded_preds)
        references.extend(decoded_refs)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [17]:
exact_matches = [1 if pred == ref else 0 for pred, ref in zip(predictions, references)]
accuracy = accuracy_score(references, predictions)

print("Evaluation Results:")
print(f"Accuracy: {accuracy:.4f}")

Evaluation Results:
Accuracy: 0.4845


In [18]:
save_path = "/content/drive/MyDrive/TEXT TO PANDAS"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/TEXT TO PANDAS/tokenizer_config.json',
 '/content/drive/MyDrive/TEXT TO PANDAS/special_tokens_map.json',
 '/content/drive/MyDrive/TEXT TO PANDAS/spiece.model',
 '/content/drive/MyDrive/TEXT TO PANDAS/added_tokens.json')